## Load the dataset


In [1]:
from VQA_Datasetv2 import VQA_Dataset, VQA_Dataset_preloaded
import clip
import torch
from torch.utils.data import DataLoader
from torch.utils.data import random_split

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model, preprocess = clip.load("ViT-B/32", device=device)
print(preprocess.__class__.__name__)
model.to(torch.float32)
print("clip model loaded")

Using cuda device
Compose
clip model loaded


## Choose one of the 3:

### 1.Normal loading

In [ ]:
loading = 3

if loading < 3:
    dataset = VQA_Dataset()
    
if loading == 0:
    dataset.load_preprocess(preprocess, device, length=6000)
elif loading == 1:
    dataset.load(preprocess, device, length=30000)
elif loading == 2:
    dataset.load_encode(preprocess, device, model, length=100)
    dataset.save("")
elif loading == 3:
    dataset.load_saved("")

### 2.Precalc

In [14]:
dataset_train = VQA_Dataset_preloaded()
dataset_val = VQA_Dataset_preloaded()
dataset_train.compute_store(preprocess, model, device, "scale_10000_", length=10000, mode="scale")# length=248348)
dataset_val.compute_store(preprocess, model, device, "scale_10000_", name="val", length=10000, mode="scale") # length=121511 )

Preprocessing Images: 100%|██████████| 10000/10000 [09:52<00:00, 16.87it/s]


### 3.Only load

In [2]:
dataset_train = VQA_Dataset_preloaded()
dataset_val = VQA_Dataset_preloaded()
dataset_train.load("full_", device, length=248348)
dataset_val.load("full_", device, name="val", length=121511)

### Create Dataloader

## New with both Validation and Training Datasets from MSCOCO

In [3]:
batch_size=2

train_size = int(len(dataset_train))
val_size = int(len(dataset_val)*0.3)
test_size = int(len(dataset_val))-val_size
print("Train size: ", train_size)
print("Test size: ", test_size)
print("Val size: ", val_size)
generator = torch.Generator().manual_seed(42)
test_dataset, val_dataset = random_split(dataset_val, [test_size, val_size], generator=generator)

train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

Train size:  248348
Test size:  85058
Val size:  36453


### Old

In [3]:
batch_size=2

train_val_size = int(len(dataset)*0.8)
train_size = int(len(dataset)*0.8*0.8)
val_size = int(len(dataset)*0.8)-train_size
test_size = int(len(dataset))-train_val_size
print("Train size: ", train_size)
print("Test size: ", test_size)
print("Val size: ", val_size)
generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size], generator=generator)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

Train size:  19200
Test size:  6000
Val size:  4800


## Simple model architectue


In [4]:
from models import VQA_Model3, VQA_Model2, VQA_Model_Precalc, VQA_Model4, VQA_Model_Blip



## Evaluate

In [5]:
import tqdm

def evaluate(model, dataloader, device, test_size, show_progress=False):
    model.eval()
    correct = 0
    if show_progress:
        pbar = tqdm.tqdm(dataloader)
    else:
        pbar = dataloader
    for i, data in enumerate(pbar):
        image = data[0].to(device)
        answer_tokens = data[1].squeeze(0).to(device)
        if dataloader.batch_size == 1:
            correct_answer = torch.tensor([int(data[3])]).to(device)
            question_tokens = data[2].squeeze(0).to(device)
        else:
            correct_answer = torch.tensor([int(x) for x in data[3]]).to(device)
            question_tokens = data[2].squeeze(1).to(device)

        with torch.no_grad():
            similarity = model(image, question_tokens, answer_tokens)
            pred = similarity.argmax(dim=-1)
            # get sum of correct answers
            correct += (pred == correct_answer).sum().item()
    print(f"Correct: {correct}, Total: {test_size}")
    return correct/(test_size)


In [8]:
combined_model = VQA_Model_Precalc(model, device)
evaluate(combined_model, test_dataloader, device, test_size, show_progress=True)

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:00<00:00, 135.97it/s]

Correct: 0, Total: 50


0.0

## Training

In [6]:
def train(model, train_dataloader, val_dataloader, device, test_size, epochs=10, patience=3, precalculated=False, save_name="model"):
    print(device)
    model.train()
    print(model.parameters())
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = torch.nn.CrossEntropyLoss()

    pbar = tqdm.tqdm(range(epochs))
    prev_acc = 0.0
    best_acc = 0.0
    count = 0
    for epoch in pbar:
        for i, data in enumerate(train_dataloader):

            image = data[0].to(device)
            answer_tokens = data[1].squeeze(0).to(device)
            if train_dataloader.batch_size == 1:
                correct_answer = torch.tensor(int(data[3])).to(device)
                question_tokens = data[2].squeeze(0).to(device)
            else:
                correct_answer = torch.tensor([int(x) for x in data[3]]).to(device)
                question_tokens = data[2].squeeze(1).to(device)
            
            optimizer.zero_grad()
            similarity = model(image, question_tokens, answer_tokens)

            # transform asnwer to tensor of the same shape as similarity before only correct index
            #one_hot_encoding = torch.zeros(similarity.shape[1])
            #one_hot_encoding[correct_answer] = 1
            #one_hot_encoding = one_hot_encoding.to(device)
            

            loss = loss_fn(similarity, correct_answer)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0, error_if_nonfinite=True)
            optimizer.step()
            #print("--------------------------------------------------")
        acc = evaluate(model, val_dataloader, device,test_size)
        if acc > best_acc:
            best_acc = acc
            # save model
            model.save(save_name)
        if acc < prev_acc:
            count += 1
        else:
            count = 0
        prev_acc = acc
        pbar.set_description(f"Epoch {epoch} loss: {loss.item()}, acc: {acc}", refresh=True)
        if count == patience:
            print("early stopping")
            break 

In [7]:
import time

trained_model = VQA_Model_Precalc(model, device)
# freeze the clip model
for param in trained_model.model.parameters():
    param.requires_grad = False


evaluate(trained_model, test_dataloader, device, test_size)
train(trained_model, train_dataloader, val_dataloader, device, val_size, epochs=500, save_name="model_test")
# evaluate the model
evaluate(trained_model, test_dataloader, device, test_size)

Correct: 3838, Total: 85058
cuda
<generator object Module.parameters at 0x0000022C26377680>


Epoch 0 loss: 0.6819792985916138, acc: 0.4962005870573067:   0%|          | 1/500 [13:16<110:22:25, 796.28s/it]

torch.Size([1, 18, 512])
Correct: 18088, Total: 36453


Epoch 1 loss: 0.6089828610420227, acc: 0.5196005815707898:   0%|          | 2/500 [20:40<81:29:12, 589.06s/it] 

torch.Size([1, 18, 512])
Correct: 18941, Total: 36453


Epoch 1 loss: 0.6089828610420227, acc: 0.5196005815707898:   0%|          | 2/500 [24:50<103:06:44, 745.39s/it]


KeyboardInterrupt: 

## Save the model!

In [ ]:
# save the model
trained_model.save("trained_model_")

## Load and evaluate


In [8]:
#load and evaluate the model
from models import VQA_Model4

trained_model = VQA_Model_Precalc(model, device) # need to choose the same model!
trained_model.load("best_model_")
#print(trained_model)
evaluate(trained_model, test_dataloader, device, test_size, show_progress=True)

100%|██████████| 3000/3000 [00:43<00:00, 68.31it/s]

Correct: 2809, Total: 6000


0.4681666666666667

In [ ]:
"""
import clip
from PIL import Image
import torch

for model_name in clip.available_models():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using {device} device and model: {model_name}")
    model, preprocess = clip.load(model_name, device=device)
    model.to(torch.float32)
    text_tokens = clip.tokenize(["question_text"]).to(device)
    image = preprocess(Image.open("CLIP.jpg")).unsqueeze(0).to(device)
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_tokens)
    print(f"text_t {text_tokens.dtype}, text_f {text_features.dtype}, image_t {image.dtype}, image_f {image_features.dtype}")
    """


'\nimport clip\nfrom PIL import Image\nimport torch\n\nfor model_name in clip.available_models():\n    device = "cuda" if torch.cuda.is_available() else "cpu"\n    print(f"Using {device} device and model: {model_name}")\n    model, preprocess = clip.load(model_name, device=device)\n    model.to(torch.float32)\n    text_tokens = clip.tokenize(["question_text"]).to(device)\n    image = preprocess(Image.open("CLIP.jpg")).unsqueeze(0).to(device)\n    image_features = model.encode_image(image)\n    text_features = model.encode_text(text_tokens)\n    print(f"text_t {text_tokens.dtype}, text_f {text_features.dtype}, image_t {image.dtype}, image_f {image_features.dtype}")\n    '